<a href="https://colab.research.google.com/github/GhislainBisamaza/Machine_learning_calibration_LCS/blob/main/Data_prep_temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta

import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
from sklearn.model_selection import train_test_split
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings("ignore")

In [ ]:
cf.set_config_file(offline = True)

In [ ]:
def null_counter(df):
    # Check for null values in the whole dataset
    print("Null values in the whole dataset:")
    print(df.isnull().sum())

    # Check for null values for each individual sensor
    sensors = df['Name'].unique()
    for sensor in sensors:
        sensor_df = df[df['Name'] == sensor]
        print(f"\nNull values for sensor {sensor}:")
        print(sensor_df.isnull().sum())

# def convert_todatetime(df):
#     """This function combines the date and time column of a datframe and creates a pandas datetime object"""
#     return pd.to_datetime(df['Date'].astype(str) + ' ' + df['Time'], format='%d.%m.%Y %H:%M')

def convert_todatetime(df):
    """This function combines the date and time column of a datframe and creates a pandas datetime object"""

    df['Date'] = df['Date'].astype(str)
    df['Time'] = df['Time'].astype(str)
    df['Date'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
    df['Date'] = df['Date'] - timedelta(hours=3)

    return df

def strip_time(df, col='Date'):
    """Covert object to datetime and strip seconds microseconds from the date object"""
    # dt = pd.to_datetime(df[col], format='mixed')
    dt = pd.to_datetime(df[col])
    df[col] = pd.to_datetime(dt.apply(lambda x: x.strftime('%Y-%m-%d %H:%M')))

    return df

def date_range(df, col):
    """Taking a dataframe return the start and end date based on a datetime column"""
    return df[col].min(), df[col].max()

def get_frequency(df, thres_mins=10):
    """This function takes a dataframe and datetime column
    determines the frequency of log values"""

    # Convert the 'Date' column to datetime data type
    df['Date'] = pd.to_datetime(df['Date'], format='mixed')

    # Calculate the time difference between consecutive rows for each sensor
    df['TimeDiff'] = df.groupby('Name')['Date'].diff()

    # Calculate the frequency of logging for each sensor
    sensor_frequency = df.groupby('Name')['TimeDiff'].mean()

    # Format the frequency times to the nearest minutes
    sensor_frequency = pd.to_timedelta(sensor_frequency).round('T')

    # Print the frequency of logging for each sensor
    print("Frequency of logging for each sensor:")
    print(sensor_frequency)

    # Return two dataframes one with values above a certain threshold and another with summary statistics
    # Get a DataFrame where the difference between logging times is more than thres_mins minutes
    filtered_df = df[df['TimeDiff'].dt.total_seconds() > (thres_mins*60)]

    # Group the DataFrame by sensor and calculate the desired statistics
    summary_df = df.groupby('Name').agg({'Value': ['count', 'sum', 'mean']})

    # Rename the columns
    summary_df.columns = ['Count', 'Sum', 'Mean']

    return summary_df, filtered_df, df

def select_data_range_values(start_date, end_date, df, col='Date', details=False):
    """This function slices off data for a given date rnage where the start and end date are specified"""
    # Slice off the dataframe matching this condition
    sliced_df = df[(df[col] >= start_date) & (df[col] <= end_date)]

    if details:
        # Check if there are some missing data
        start, end = date_range(sliced_df, col)

        if start != start_date:
            print(f'Mismatch start: Expected {start_date} vs Gotten (from slice) {start}')
        if end != end_date:
            print(f'Mismatch end: Expected {end_date} vs Gotten (from slice) {end}')

    return sliced_df


def smooth_data(df, avg='5'):
    """This function takes a dataframe object and computes 5 minutes average which is based on the frequency of the reference data
    it returns this smoothed average"""

    # set the date column as the index
    df = df.set_index('Date')

    # calculate the average over 5 minutes
    df_avg = df['Value'].resample(f'{avg}T').mean()

    # Reset the index
    df_avg = df_avg.reset_index()

    # return the averaged df
    return df_avg


def precise_frequency(df, col='Date'):
    # calculate time difference between consecutive datetimes
    time_diffs = df[col].diff()

    # Get the frequency of measuremeant as a mean
    sensor_frequency = time_diffs.mean()

    # count the number of occurrences of each time difference
    freq_counts = time_diffs.value_counts()
    print(f"Frequency counts \n{freq_counts}")

    return pd.to_timedelta(sensor_frequency).round('T'), freq_counts


def plot_date_range(combined_df,start_date,end_date,features_list,title,unit,show_diff=False):
    """
    This function helps to plot Sensors data for a specific data range and their difference.
    it can be used following this example:
    temperature_data_df.loc[start_date:end_date,['hdc_1', 'hdc_2','hdc_average',"temperature S001544 (degrees Celsius)"]].iplot(title= "Monthly HDC sensor temperature data", xTitle= "Time", yTitle= "degrees Celsius")
    """
    if(show_diff):
        return combined_df.loc[start_date:end_date,features_list].iplot( kind = "spread",
                                                                        title="{} Sensors Data from {} to {} and their Difference".format(title, start_date,end_date),
                                                                        xTitle="Time",
                                                                        yTitle= unit)

    return combined_df.loc[start_date:end_date,features_list].iplot( title="{} Sensors Data from {} to {}".format(title, start_date,end_date),
                                                                     xTitle="Time",
                                                                     yTitle= unit)

In [ ]:
temperature_df = pd.read_excel("WeatherStation Data - 20-06-2023.xlsx", sheet_name='Temperature')
temperature_df = convert_todatetime(temperature_df)
temperature_df

In [ ]:
sensors_list = temperature_df["Name"].unique().tolist()
for sensor in sensors_list:
    new_dataframe =  temperature_df.loc[temperature_df["Name"] == sensor]
    new_dataframe = new_dataframe.set_index("Date")
    new_dataframe = new_dataframe.sort_index()
    new_dataframe["Value"].iplot(title="{} Sensor data".format(sensor))

In [ ]:
# Temperature data information

print("Temperature data information")
print("============================")

# Get date ranges
temp_start, temp_end = date_range(temperature_df, 'Date')
print(f'Start date: {temp_start} \nEnd date: {temp_end}')

# Get the frequency of measurement
freq = get_frequency(temperature_df)

In [ ]:
freq[1]

In [ ]:
freq[2]['TimeDiff'].iplot()

In [ ]:
# humidity and temperature have the same sensor
# group by sensor and create a dictionary of dataframes

# temp_sensor_dfs = {sensor: group for sensor, group in sliced_temp.groupby('Name')}

# # assert the lengths of extracted dataframes is same as the unique sensors we have
# assert len(temp_sensor_dfs) == len(sliced_temp['Name'].unique())

# # visualize a sample df [fun gets the name from a sample]
# temp_sensor_dfs[sliced_temp.sample(1).values[0,3]]


#second_approach

temp_sensor_dfs = {sensor: group for sensor, group in sliced_temp.groupby('Name')}
assert len(temp_sensor_dfs) == len(sliced_temp['Name'].unique())

sample_row = sliced_temp.sample(1)
key = sample_row.values[0, 3]

if key in temp_sensor_dfs:
    selected_group = temp_sensor_dfs[key]
    print(selected_group)
else:
    print("Key does not exist in temp_sensor_dfs dictionary.")

In [ ]:
# Smooth data for all temperature sensors
for sensor in temp_sensor_dfs.keys():
    temp_sensor_dfs[sensor] = smooth_data(temp_sensor_dfs[sensor])
    precise_frequency(temp_sensor_dfs[sensor])

In [ ]:
# Combine the temperature reference and the temperature sensors data to form one single dataframe
temperature_data_df = temp_hum_ref[["timestamp (UTC)","temperature S001544 (degrees Celsius)","humiditysensortemperature S001544 (degrees Celsius)"]].reset_index(drop=True)
temperature_data_df= temperature_data_df.rename(columns={'timestamp (UTC)': 'Date'})

for sensor in temp_sensor_dfs.keys():
    new_df = temp_sensor_dfs[sensor][["Date", "Value"]]

    temperature_data_df = temperature_data_df.merge(new_df, on="Date",how='outer')
    temperature_data_df = temperature_data_df.rename(columns={"Value": sensor})

temperature_data_df = temperature_data_df.set_index("Date")
temperature_data_df

In [ ]:
temperature_data_df["bme_average"] = temperature_data_df[['bme_1', 'bme_2', 'bme_3']].mean(axis=1)
temperature_data_df["hdc_average"] = temperature_data_df[['hdc_1', 'hdc_2']].mean(axis=1)
temperature_data_df["htu_average"] = temperature_data_df[['htu_1', 'htu_2', 'htu_3']].mean(axis=1)
temperature_data_df["sht_average"] = temperature_data_df[['sht_1', 'sht_2', 'sht_3']].mean(axis=1)

In [ ]:
temp_ref_available_data = len(temperature_data_df["temperature S001544 (degrees Celsius)"].dropna())
average_sensors_data = ["bme_average","hdc_average","htu_average","sht_average"]
print("Percentage of available data for each Temperature sensor type compared to the Reference Data")
print("================================================================================")
print("")
for average in average_sensors_data:
    unique_sensor_data = len(temperature_data_df[average].dropna())
    percentage = unique_sensor_data*100//temp_ref_available_data
    print("The available {} sensor data is {} %".format(average,percentage))

In [ ]:
start_period_temp = temperature_data_df.index.min()
end_period_temp = temperature_data_df.index.max()

# Long periods
start_date = '2021-12-3'
end_date = '2022-01-16'

# shorter periods
short_start_date = '2021-12-16'
short_end_date = '2021-12-18'

# feature list
all_features = temperature_data_df.columns.tolist()
all_features